In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

import joblib
import json
import warnings
warnings.simplefilter("ignore")

# I. Load the backtested trading data

In [2]:
# load data
raw_data = pd.read_csv('trades_data.csv')
raw_data.head()

,EntryPrice,win_loss,long_short,entry_month,entry_date,entry_day,entry_hour
0,1.3566,loss,short,4,23,0,8
1,1.3566,loss,short,4,23,0,8
2,1.3568,loss,short,4,24,1,0
3,1.3568,loss,short,4,24,1,0
4,1.3568,loss,short,4,24,1,0


In [3]:
# data decription
raw_data.describe()

,EntryPrice,entry_month,entry_date,entry_day,entry_hour
count,2443.000000,2443.000000,2443.000000,2443.000000,2443.000000
mean,1.259341,6.767090,15.620958,2.109701,10.167826
std,0.131127,3.453237,8.706682,1.635073,5.874165
min,1.039650,1.000000,1.000000,0.000000,0.000000
25%,1.133285,4.000000,8.000000,1.000000,8.000000
50%,1.256750,7.000000,15.000000,2.000000,12.000000
75%,1.359370,10.000000,23.000000,3.000000,16.000000
max,1.592810,12.000000,31.000000,6.000000,20.000000


# II. Preprocessing

## II.I. define dataframe for inferencing

In [4]:
# Define how many inferences we want to generate
inf_count =  round(raw_data.shape[0]*0.05)
inf_count

122

In [5]:
# Get Data for model inference
data_inf = raw_data.sample(inf_count, random_state=33)
data_inf.sample(5)

,EntryPrice,win_loss,long_short,entry_month,entry_date,entry_day,entry_hour
523,1.34612,win,long,4,5,0,0
2104,1.10620,win,short,11,21,3,16
739,1.33553,loss,short,10,5,2,12
104,1.43982,win,long,12,24,0,12
2323,1.20664,loss,short,4,28,2,4


## II.II. Split data

In [6]:
# Copy the raw_data
raw_data_1 = raw_data.copy()

In [7]:
# define features variable as X
X = raw_data_1.drop('win_loss', axis=1)

In [8]:
# define target variable as y
y = raw_data_1['win_loss']
y.sample(5)

1396    loss
830      win
1769    loss
27       win
1470    loss
Name: win_loss, dtype: object

In [9]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.075, random_state=33, stratify=y)

print(f'Train size: {X_train.shape[0]}')
print(f'Test size: {X_test.shape[0]}')

Train size: 2259
Test size: 184


## II.III. Handling empty values

In [10]:
# sum null value
X_train.isnull().sum()

EntryPrice     0
long_short     0
entry_month    0
entry_date     0
entry_day      0
entry_hour     0
dtype: int64

## II.IV. Feature Scaling

In [11]:
# Get Numerical Columns and Categorical Columns

num_columns = X_train.select_dtypes(include=np.number).columns.tolist()
cat_columns = X_train.select_dtypes(include=['object']).columns.tolist()

print('Numerical Columns   : ', num_columns)
print('Categorical Columns : ', cat_columns)

Numerical Columns   :  ['EntryPrice', 'entry_month', 'entry_date', 'entry_day', 'entry_hour']
Categorical Columns :  ['long_short']


In [12]:
X_train_2 = X_train.copy()

In [13]:
# # Define scaler using min max scaler
# scaler = MinMaxScaler()

# # fir and transform scaling
# X_train_scaled = scaler.fit_transform(X_train_2[num_columns])
# X_test_scaled = scaler.transform(X_test[num_columns])

In [14]:
# Define scaler using standard scaler
scaler = StandardScaler()

# fir and transform scaling
X_train_scaled = scaler.fit_transform(X_train_2[num_columns])
X_test_scaled = scaler.transform(X_test[num_columns])

## II.V. Feature Encoding

In [15]:
# Define Encoder
encoder = OneHotEncoder(sparse=False)

# fit and transform scaling
X_train_encoded = encoder.fit_transform(X_train_2[cat_columns])
X_test_encoded = encoder.transform(X_test[cat_columns])

In [16]:
# Summarize new encoded data
X_test_encoded

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [17]:
# concat encoded and scaled training dataset
X_train_fin = np.concatenate((X_train_encoded, X_train_scaled), axis=1 )

In [18]:
# concat encoded and scaled test dataset
X_test_fin = np.concatenate((X_test_encoded, X_test_scaled), axis=1 )

# III. Model

In [19]:
# Define model
model = RandomForestClassifier()

In [20]:
# fit model
model.fit(X_train_fin, y_train)

RandomForestClassifier()

In [21]:
# Predict the dataset
y_train_pred=model.predict(X_train_fin)
y_test_pred=model.predict(X_test_fin)

In [22]:
# Evaluate training model
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

        loss       0.92      0.94      0.93      1393
         win       0.90      0.87      0.89       866

    accuracy                           0.91      2259
   macro avg       0.91      0.91      0.91      2259
weighted avg       0.91      0.91      0.91      2259



In [23]:
# Evaluate test model
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

        loss       0.72      0.84      0.77       114
         win       0.64      0.46      0.53        70

    accuracy                           0.70       184
   macro avg       0.68      0.65      0.65       184
weighted avg       0.69      0.70      0.68       184



# IV. Inferences

## IV.I. Saving models

In [24]:
with open('model.pkl', 'wb') as file_1:
  joblib.dump(model, file_1)

with open('model_scaler.pkl', 'wb') as file_2:
  joblib.dump(scaler, file_2)

with open('model_encoder.pkl', 'wb') as file_3:
  joblib.dump(encoder, file_3)

with open('list_num.txt', 'w') as file_4:
  json.dump(num_columns, file_4)

with open('list_cat.txt', 'w') as file_5:
  json.dump(cat_columns, file_5)

## IV.II. Preprocessing inferences

In [25]:
# Summarize inference dataset
data_inf.sample(3)

,EntryPrice,win_loss,long_short,entry_month,entry_date,entry_day,entry_hour
423,1.42397,win,long,8,19,2,12
1685,1.07869,win,short,3,29,2,4
2351,1.19262,loss,long,6,24,3,8


In [26]:
# Reseting index
data_inf.reset_index(inplace=True)

In [27]:
# Encoding and scaling the dataset
inf_scaled = scaler.transform(data_inf[num_columns])
inf_encoded = encoder.transform(data_inf[cat_columns])

In [28]:
# Concat encoded and scaled dataset
X_inf = np.concatenate((inf_encoded, inf_scaled), axis=1 )
X_inf

array([[ 0.        ,  1.        ,  0.72129468, -0.52142605, -0.86992188,
        -0.67447028, -0.36582621],
       [ 1.        ,  0.        , -1.53651169, -1.390726  ,  0.85218497,
         0.54883631,  1.66797543],
       [ 0.        ,  1.        , -1.37411425, -1.10095935,  1.54102771,
        -0.06281698, -1.04376009],
       [ 1.        ,  0.        ,  0.61565603,  0.34787391, -0.52550051,
        -1.28612358, -0.36582621],
       [ 0.        ,  1.        ,  0.4732236 ,  1.50694051,  0.62257072,
         1.16048961, -1.72169397],
       [ 0.        ,  1.        , -0.58882355,  1.21717386, -0.64030763,
        -0.67447028, -0.36582621],
       [ 0.        ,  1.        , -0.63762693,  0.34787391, -0.06627202,
        -0.67447028, -1.04376009],
       [ 1.        ,  0.        , -0.90153233,  1.21717386,  0.73737784,
         0.54883631, -1.72169397],
       [ 0.        ,  1.        , -0.98621155,  0.63764056,  0.04853511,
        -0.06281698, -0.36582621],
       [ 1.        ,  0.    

In [29]:
# predict the dataset
inf_pred = model.predict(X_inf)
inf_pred

array(['win', 'win', 'win', 'loss', 'loss', 'loss', 'loss', 'loss',
       'loss', 'win', 'loss', 'win', 'loss', 'win', 'loss', 'loss',
       'loss', 'loss', 'loss', 'win', 'loss', 'loss', 'loss', 'loss',
       'loss', 'win', 'win', 'win', 'win', 'loss', 'loss', 'loss', 'loss',
       'win', 'loss', 'loss', 'loss', 'win', 'loss', 'loss', 'win', 'win',
       'loss', 'loss', 'loss', 'win', 'loss', 'loss', 'loss', 'win',
       'loss', 'loss', 'loss', 'loss', 'loss', 'win', 'loss', 'loss',
       'win', 'loss', 'loss', 'loss', 'win', 'win', 'win', 'loss', 'loss',
       'win', 'win', 'win', 'loss', 'win', 'win', 'win', 'win', 'loss',
       'loss', 'win', 'win', 'loss', 'win', 'loss', 'win', 'win', 'loss',
       'win', 'loss', 'win', 'loss', 'loss', 'loss', 'loss', 'loss',
       'win', 'loss', 'loss', 'loss', 'win', 'loss', 'loss', 'win', 'win',
       'loss', 'loss', 'loss', 'loss', 'loss', 'loss', 'loss', 'win',
       'loss', 'win', 'win', 'loss', 'loss', 'loss', 'loss', 'loss',
 

In [30]:
# Evaluate test model
print(classification_report(data_inf['win_loss'],inf_pred))

              precision    recall  f1-score   support

        loss       0.87      0.97      0.92        68
         win       0.96      0.81      0.88        54

    accuracy                           0.90       122
   macro avg       0.91      0.89      0.90       122
weighted avg       0.91      0.90      0.90       122



In [31]:
# Evaluate test model
print(precision_score(data_inf['win_loss'], inf_pred, pos_label='win'))

0.9565217391304348
